2018 / 1 / 31

Wayne Nixalo

# RetinaNet Finetuning -- Dev 4

In [1]:
import keras
import keras.preprocessing.image

# import keras_retinanet.losses
# import keras_retinanet.layers
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
# from keras_retinanet.callbacks import RedirectModel
# from keras_retinanet.models.resnet import ResNet50RetinaNet
# from keras_retinanet.utils.keras_version import check_keras_version
from keras_retinanet.utils.image import preprocess_image, resize_image
from keras_retinanet.models.resnet import custom_objects # <-- what do I need this for?

import tensorflow as tf

import numpy as np
import os

/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

In [3]:
keras.backend.tensorflow_backend.set_session(get_session())

model = keras.models.load_model('data/retinanet-model/resnet50_coco_best_v1.2.2.h5',
                                    custom_objects=custom_objects)

/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/keras/models.py:271: UserWarning: Output "non_maximum_suppression_1" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "non_maximum_suppression_1" during training.
  sample_weight_mode=sample_weight_mode)


## Removing Top Layer from Retinanet

In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
padding_conv1 (ZeroPadding2D)   (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9408        padding_conv1[0][0]              
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_relu

In [7]:
model.layers[0]

In [14]:
for i,λ in enumerate(model.layers):
    print(f'{i}:  {λ}')

0:  <keras.engine.topology.InputLayer object at 0x18168ce198>
1:  <keras.layers.convolutional.ZeroPadding2D object at 0x18168ce208>
2:  <keras.layers.convolutional.Conv2D object at 0x18168ce390>
3:  <keras_resnet.layers._batch_normalization.BatchNormalization object at 0x18168ce4e0>
4:  <keras.layers.core.Activation object at 0x18168ce6d8>
5:  <keras.layers.pooling.MaxPooling2D object at 0x18168ce668>
6:  <keras.layers.convolutional.Conv2D object at 0x18168ce828>
7:  <keras_resnet.layers._batch_normalization.BatchNormalization object at 0x18168ce978>
8:  <keras.layers.core.Activation object at 0x18168ceac8>
9:  <keras.layers.convolutional.ZeroPadding2D object at 0x18168ceb00>
10:  <keras.layers.convolutional.Conv2D object at 0x18168ceb70>
11:  <keras_resnet.layers._batch_normalization.BatchNormalization object at 0x18168cecc0>
12:  <keras.layers.core.Activation object at 0x18168cee10>
13:  <keras.layers.convolutional.Conv2D object at 0x18168cee48>
14:  <keras.layers.convolutional.Conv2

In [26]:
model.layers[-6:]

So the last 6 layers are the ones I want to cut off. I guess I'm going to want to use the keras_retinanet API to put the RegressBoxes layer back in.

In [28]:
for i in range(6):
    model.layers.pop()

In [29]:
model.layers[-6:]

Looks good. Now to the layers back on with the Functional API. The layers I'm looking to replace:

```
209:  <keras.engine.training.Model object at 0x18169ee940>
210:  <keras.layers.merge.Concatenate object at 0x1816a30048>
211:  <keras.layers.merge.Concatenate object at 0x1816a30080>
212:  <keras_retinanet.layers._misc.RegressBoxes object at 0x1816ab0e48>
213:  <keras.layers.merge.Concatenate object at 0x1816ab0ef0>
214:  <keras_retinanet.layers._misc.NonMaximumSuppression object at 0x1816a300b8>
```

Just saying right now... I don't know how I'm going to be able to replace that `keras.engine.training.Model` layer, "classification_submodel (Model)". Maybe there's a way to replace individual layers, and recompile so Keras will figure out how many in/out pars they should have? We'll see.

In [44]:
from keras.models import Model
from keras.layers import Input, Dense

a = Input(shape=(32,))
b = Dense(32)(a)
model = Model(inputs=a, outputs=b)

In [46]:
# # x = keras.engine.training.Model(model.output_shape, 4)(model)
# last = model.layers[-1].output
# x = keras.engine.training.Model(1,4)(last)

x = keras.engine.training.Model(1,4)(model)

TypeError: Input tensors to a Model must be Keras tensors. Found: 1 (missing Keras metadata).

In [34]:
last

<tf.Tensor 'regression_1/concat:0' shape=(?, ?, 4) dtype=float32>

Okay instead of messing with all this and rebuilding retinanet from scratch just to tweak a part of it, how is it done in the [Boring Detector](https://github.com/lexfridman/boring-detector/blob/master/train.py)?

In [4]:
import keras_retinanet.losses
import keras_retinanet.layers
from keras_retinanet.callbacks import RedirectModel
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
from keras_retinanet.models.resnet import ResNet50RetinaNet
from keras_retinanet.utils.keras_version import check_keras_version

In [5]:
from keras_retinanet.models.resnet import ResNet50RetinaNet
def create_models(num_classes):
    # create "base" model (no NMS)
    image = keras.layers.Input((None, None, 3))
    
    model = ResNet50RetinaNet(image, num_classes=num_classes, weights='imagenet', nms=False)
    training_model = model
    
    # append NMS for prediction only
    classification   = model.outputs[1]
    detections       = model.outputs[2]
    boxes            = keras.layers.Lambda(lambda x: x[:, :, :4])(detections)
    detections       = keras_retinanet.layers.NonMaximumSuppression(name='nms')([boxes, classification, detections])
    prediction_model = keras.models.Model(inputs=model.inputs, outputs=model.outputs[:2] + [detections])
    
    # compile model
    training_model.compile(
        loss={
            'regression'    : keras_retinanet.losses.smooth_l1(),
            'classification':keras_retinanet.losses.focal()
        },
        optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001)
    )
    
    return model, training_model, prediction_model

In [61]:
create_models(1)

/Users/WayNoxchi/Miniconda3/envs/aersu/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Output "concatenate_16" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "concatenate_16" during training.


(<keras.engine.training.Model at 0x187a590f60>,
 <keras.engine.training.Model at 0x187a494fd0>)

In [62]:
# just pulling the training model
m1= create_models(1)[0]

/Users/WayNoxchi/Miniconda3/envs/aersu/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Output "concatenate_18" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "concatenate_18" during training.


In [63]:
m1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           (None, None, None, 3 0                                            
__________________________________________________________________________________________________
padding_conv1 (ZeroPadding2D)   (None, None, None, 3 0           input_26[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9408        padding_conv1[0][0]              
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_relu

 Well.... *THAT* was simple.
 
 So now I'll see if it'll train (freeze Conv layers, train top) and then I'll take a look of whether I want to include NMS or not. NMS is included in the version of RetNet I was using, but I wasn't training it, noly predicted. This here doesn't use NMS during training. Hmm.

## Trying to train the 1-class model

In [6]:
m0,m1,m2 = create_models(1)

102948864/102945312 [==============================] - 39s 0us/step


/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Output "concatenate_2" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "concatenate_2" during training.


In [7]:
# model?
m0.output_shape

[(None, None, 4), (None, None, 1), (None, None, 5)]

In [69]:
# training model
m1.output_shape

[(None, None, 4), (None, None, 1), (None, None, 5)]

In [70]:
# prediction model
m2.output_shape

[(None, None, 4), (None, None, 1), (None, None, 5)]

In [71]:
# model?
m0.layers[-6:]

In [72]:
# training model
m1.layers[-6:]

In [73]:
# prediction model
m2.layers[-6:]

In [9]:
image_gen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True)
datagen = CSVGenerator(csv_data_file='data/interstage_retinanet_data.csv',
                       csv_class_file='data/interstage_retinanet_classes.csv',
                       image_data_generator=image_gen,
                       batch_size=1)

In [11]:
m1.fit_generator(generator = datagen,
                    steps_per_epoch = 1000, # eh?
                    epochs    = 1,
                    verbose   = 1,)

Epoch 1/1
  69/1000 [=>............................] - ETA: 13:49 - loss: 5.0982 - regression_loss: 3.9909 - classification_loss: 1.1073

/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/keras_retinanet-0.0.1-py3.6.egg/keras_retinanet/preprocessing/generator.py:100: UserWarning: Image with id 4399 (shape (400, 400, 3)) contains the following invalid boxes: [array([  0.,  17., 401., 395.,   0.])].
  [annotations[invalid_index, :] for invalid_index in invalid_indices]


 102/1000 [==>...........................] - ETA: 13:07 - loss: 4.8930 - regression_loss: 3.8575 - classification_loss: 1.0355

/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/keras_retinanet-0.0.1-py3.6.egg/keras_retinanet/preprocessing/generator.py:100: UserWarning: Image with id 4017 (shape (400, 400, 3)) contains the following invalid boxes: [array([130.,  84., 288., 401.,   0.])].
  [annotations[invalid_index, :] for invalid_index in invalid_indices]


 155/1000 [===>..........................] - ETA: 12:12 - loss: 4.6413 - regression_loss: 3.7589 - classification_loss: 0.8825

...

 866/1000 [========================>.....] - ETA: 1:54 - loss: 3.2899 - regression_loss: 2.8400 - classification_loss: 0.4499

/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/keras_retinanet-0.0.1-py3.6.egg/keras_retinanet/preprocessing/generator.py:100: UserWarning: Image with id 5991 (shape (400, 400, 3)) contains the following invalid boxes: [array([145., 140., 235., 402.,   0.])].
  [annotations[invalid_index, :] for invalid_index in invalid_indices]


 877/1000 [=========================>....] - ETA: 1:44 - loss: 3.2769 - regression_loss: 2.8281 - classification_loss: 0.4488

/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/keras_retinanet-0.0.1-py3.6.egg/keras_retinanet/preprocessing/generator.py:100: UserWarning: Image with id 4757 (shape (400, 400, 3)) contains the following invalid boxes: [array([ 66.,  40., 344., 401.,   0.])].
  [annotations[invalid_index, :] for invalid_index in invalid_indices]


1000/1000 [==============================] - 850s 850ms/step - loss: 3.1863 - regression_loss: 2.7416 - classification_loss: 0.4446


Well Ho-Lee-Gahd-Dayum.

Wait is it including the class-id along with the bounding box coordinates?

In [13]:
import cv2

In [33]:
test_image_path = 'data/interstage_train/000000-000412/000001.jpg'
test_image = cv2.imread(test_image_path)

# prediction = m1.predict_on_batch(np.array([test_image]))
prediction = m1.predict_on_batch(np.expand_dims(test_image, axis=0))

In [38]:
prediction[0].shape

(1, 30231, 4)

In [24]:
retinamodel = keras.models.load_model('data/retinanet-model/resnet50_coco_best_v1.2.2.h5',
                                    custom_objects=custom_objects)

/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/keras/models.py:271: UserWarning: Output "non_maximum_suppression_2" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "non_maximum_suppression_2" during training.
  sample_weight_mode=sample_weight_mode)


In [39]:
prediction2 = retinamodel.predict_on_batch(np.expand_dims(test_image, axis=0))

In [42]:
prediction2[2].shape

(1, 300, 84)

In [96]:
# preprocess image for neural network
image = preprocess_image(test_image)
image, scale = resize_image(image)

# detect on image

# ? model
# _,_,detections = m0.predict_on_batch(np.expand_dims(image, axis=0))
# training model
# _,_,detections = m1.predict_on_batch(np.expand_dims(image, axis=0))
# prediction model
_,_,detections = m2.predict_on_batch(np.expand_dims(image, axis=0))
# RetinaNet -- MS COCO dataset pretrained
# _,_,detections = model.predict_on_batch(np.expand_dims(image, axis=0))

# compute predicted labels and scores
detected_label = np.argmax(detections[0,:,4:], axis=1)
scores         = detections[0, np.arange(detections.shape[1]), 4 + detected_label]

# correct for image scale
detections[0,:,:4] /= scale

I have an issue here. It looks like when I initialize RetinaNet for 1 class, it's trying to predict for around about 30,000 or 60,000 predictions and scores.. Why..?

Okay. How's this. What if I re-implement the Boring Detector, and then repurpose that as a pilot detector? And why did I see no mention of RetinaNets initialization API in their own documentation? They just didn't doc it? Hmm..

Oh. *Oh*. ***OHHHhhhhh***. Okay. *phew*. ***That's*** what happens when you use the training model instead of the predictions model for predicting. Now that makes sense.

I mean, I don't exactly know why, but I see it working as advertised now.

**Stock RetinaNet (from before, pretrained on MS COCO dataset):**

In [85]:
detections.shape

(1, 300, 84)

In [86]:
scores.shape

(300,)

In [87]:
bbx = detections[0, 0, :4]
bbx

array([117.726906,  61.761536, 276.647   , 353.71005 ], dtype=float32)

**1-Class Initialized RetinaNet ?-Model (pretrained on ImageNet):**

In [89]:
detections.shape

(1, 67995, 5)

In [90]:
scores.shape

(67995,)

In [91]:
bbx = detections[0, 0, :4]
bbx

array([-12.51579 ,  -5.019752,  17.714884,  10.247435], dtype=float32)

**1-Class Initialized RetinaNet Training-Model (pretrained on ImageNet):**

In [93]:
detections.shape

(1, 67995, 5)

In [94]:
scores.shape

(67995,)

In [95]:
bbx = detections[0, 0, :4]
bbx

array([-12.51579 ,  -5.019752,  17.714884,  10.247435], dtype=float32)

**1-Class Initialized RetinaNet Predictions-Model (pretrained on ImageNet):**

In [97]:
detections.shape

(1, 300, 5)

In [98]:
scores.shape

(300,)

In [99]:
bbx = detections[0, 0, :4]
bbx

array([117.78066 ,  87.328606, 267.3493  , 383.69162 ], dtype=float32)

Alright looking good. I dont remember if training the training-model affected the prediction-model (of course it would, why wouldn't it? Though I don't understand the mechanics of that yet... both model share memory?) and the numbers are close-ish.

The 'pilot' prediction in the first image `../000000.jpg` is very similar to the 'person' class in stock retinanet, so I'd expect the prediced bounding boxes to be similar to each other.

This is.... exciting. Wow. Yeah. This means I have a working method to finetune RetinaNet and get on with this project. -- testing/debugging included.

Alright! Onwards I gotta clean this all up; I think I'll have it all in a `.py` file instead of a `.ipynb`, and I can easily just make one if I feel I need to / for display purposes. Great.

2018/1/31 18:43

WNixalo


**NOTE**: I don't know if the `invalid boxes` warning/error thing is going to affect things. We'll see. This test case seemed to work fine.

In [ ]:
model.layers.pop()
for layer in model.layers: layer.trainable = False
    
# recover the output from the last layer in the model and use as input to new Dense layer
last = model.layers[-1].output
x = Dense(train_batches.num_class, activation="softmax")(last)
model = Model(model.input, x)

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])